# **Predicting Disaster Tweets Using LSTM**

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

import re
text_cleaning_regex = "@S+|https?:S+|http?:S|[^A-Za-z0-9]+"

import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import nltk
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

from keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

import tensorflow as tf
import keras

from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Dropout, Input, SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

from sklearn import model_selection, metrics, preprocessing, ensemble, model_selection, metrics


from tensorflow import keras
from tensorflow.keras import layers
from sklearn import model_selection
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Download Data

In [ ]:
# !wget https://github.com/Laterma/Predicting-Disaster-Tweets-Using-LSTM/blob/main/Predicting_Disaster_Tweets.rar
!git clone https://github.com/Laterma/Predicting-Disaster-Tweets-Using-LSTM.git

fatal: destination path 'Predicting-Disaster-Tweets-Using-LSTM' already exists and is not an empty directory.


In [ ]:
# save the model to disk
filename = 'Predicting-Disaster-Tweets-Using-LSTM/lstm_model.sav'
# pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

# Loading Dataset

In [ ]:
df_test = pd.read_csv('Predicting-Disaster-Tweets-Using-LSTM/test.csv')

#Preparing Data

In [ ]:
# df_test = df_test.drop(['location', 'keyword', 'id'], axis=1)

In [ ]:
df_test.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [ ]:
df_test = df_test.drop(['location', 'keyword', 'id'], axis=1)
df_test.head(10)

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
5,We're shaking...It's an earthquake
6,They'd probably still show more life than Arse...
7,Hey! How are you?
8,What a nice hat?
9,Fuck off!


In [ ]:
df_test['cleaned_text'] = df_test['text']

In [ ]:
df_test[['text', 'cleaned_text']]

,text,cleaned_text
0,Just happened a terrible car crash,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s...","Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are...","there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,Storm in RI worse than last hurricane. My city...,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...,MEG issues Hazardous Weather Outlook (HWO) htt...


In [ ]:
def clean_tweets(tweet):
  
  #remove uni-code characters and lower case the tweets
  tweet = re.sub(text_cleaning_regex, ' ', str(tweet).lower()).strip()

  # remove numbers from text
  tweet = re.sub(r'\d+', '', tweet)

  #stemming
  tweet = stemmer.stem(tweet)

  #remove stop words
  stop = stopwords.words('english')
  tweet = " ".join([word for word in tweet.split() if word not in (stop)])

  #apply tokenization
  tweet = re.split('\W+',tweet)

  # return " ".join(" ".join(tweet).split())
  return tweet

df_test['cleaned_text'] = df_test['text'].apply(lambda x: clean_tweets(x) )



In [ ]:
df_test[['text', 'cleaned_text']]

,text,cleaned_text
0,Just happened a terrible car crash,"[happened, terrible, car, crash]"
1,"Heard about #earthquake is different cities, s...","[heard, earthquake, different, cities, stay, s..."
2,"there is a forest fire at spot pond, geese are...","[forest, fire, spot, pond, geese, fleeing, acr..."
3,Apocalypse lighting. #Spokane #wildfires,"[apocalypse, lighting, spokane, wildfir]"
4,Typhoon Soudelor kills 28 in China and Taiwan,"[typhoon, soudelor, kills, china, taiwan]"
...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,"[earthquake, safety, los, angeles, safety, fas..."
3259,Storm in RI worse than last hurricane. My city...,"[storm, ri, worse, last, hurricane, city, amp,..."
3260,Green Line derailment in Chicago http://t.co/U...,"[green, line, derailment, chicago, http, co, u..."
3261,MEG issues Hazardous Weather Outlook (HWO) htt...,"[meg, issues, hazardous, weather, outlook, hwo..."


# Pre-Building the model

In [ ]:
df_test['cleaned_text']

0                        [happened, terrible, car, crash]
1       [heard, earthquake, different, cities, stay, s...
2       [forest, fire, spot, pond, geese, fleeing, acr...
3                [apocalypse, lighting, spokane, wildfir]
4               [typhoon, soudelor, kills, china, taiwan]
                              ...                        
3258    [earthquake, safety, los, angeles, safety, fas...
3259    [storm, ri, worse, last, hurricane, city, amp,...
3260    [green, line, derailment, chicago, http, co, u...
3261    [meg, issues, hazardous, weather, outlook, hwo...
3262    [cityofcalgary, activated, municipal, emergenc...
Name: cleaned_text, Length: 3263, dtype: object

In [ ]:
tokenizer.fit_on_texts(df_test['cleaned_text'])
word_index = tokenizer.word_index
print(word_index)

{'co': 1, 'http': 2, 'https': 3, 'amp': 4, 'like': 5, 'new': 6, 'get': 7, 'fire': 8, 'via': 9, 'news': 10, 'one': 11, 'people': 12, 'would': 13, 'emergency': 14, 'attack': 15, 'video': 16, 'first': 17, 'rt': 18, 'suicide': 19, 'police': 20, 'disaster': 21, 'u': 22, 'us': 23, 'hiroshima': 24, 'still': 25, 'crash': 26, 'watch': 27, 'fires': 28, 'w': 29, 'storm': 30, 'today': 31, 'time': 32, 'full': 33, 'two': 34, 'got': 35, 'burning': 36, 'body': 37, 'year': 38, 'going': 39, 'go': 40, 'bomb': 41, 'know': 42, 'day': 43, 'last': 44, 'see': 45, 'nuclear': 46, 'make': 47, 'world': 48, 'think': 49, 'old': 50, 'love': 51, 'back': 52, 'dead': 53, 'need': 54, 'car': 55, 'forest': 56, 'life': 57, 'right': 58, 'even': 59, 'pm': 60, 'man': 61, 'california': 62, 'water': 63, 'th': 64, 'bombing': 65, 'buildings': 66, 'never': 67, 'wild': 68, 'post': 69, 'say': 70, 'best': 71, 'god': 72, 'train': 73, 'mass': 74, 'let': 75, 'could': 76, 'bomber': 77, 'way': 78, 'stop': 79, 'services': 80, 'army': 81, '

In [ ]:
vocab_size = len(word_index) + 1
print("Vocabulary Size :", vocab_size)

Vocabulary Size : 11922


In [ ]:
test = pad_sequences(tokenizer.texts_to_sequences(df_test['cleaned_text']),maxlen = 30)

In [ ]:
test

array([[    0,     0,     0, ...,   951,    55,    26],
       [    0,     0,     0, ...,   600,   952,  2349],
       [    0,     0,     0, ...,  1126,   308,   601],
       ...,
       [    0,     0,     0, ...,     2,     1, 11920],
       [    0,     0,     0, ...,     2,     1, 11921],
       [    0,     0,     0, ...,    14,   157,  2248]], dtype=int32)

In [ ]:
df_test['prediction'] = model.predict(test, verbose=1, batch_size=30)


109/109 [==============================] - 3s 16ms/step


In [ ]:
df_test['cleaned_text']

0                        [happened, terrible, car, crash]
1       [heard, earthquake, different, cities, stay, s...
2       [forest, fire, spot, pond, geese, fleeing, acr...
3                [apocalypse, lighting, spokane, wildfir]
4               [typhoon, soudelor, kills, china, taiwan]
                              ...                        
3258    [earthquake, safety, los, angeles, safety, fas...
3259    [storm, ri, worse, last, hurricane, city, amp,...
3260    [green, line, derailment, chicago, http, co, u...
3261    [meg, issues, hazardous, weather, outlook, hwo...
3262    [cityofcalgary, activated, municipal, emergenc...
Name: cleaned_text, Length: 3263, dtype: object

In [ ]:
df_test['prediction']

0       0.001021
1       0.558098
2       0.002652
3       0.000146
4       0.004052
          ...   
3258    0.071266
3259    0.999603
3260    0.035815
3261    0.039084
3262    0.036640
Name: prediction, Length: 3263, dtype: float32

In [ ]:
def predict_tweet(score):
    return 1 if score>0.5 else 0

In [ ]:
df_test['prediction'] = df_test['prediction'].apply(lambda x: predict_tweet(x) )

In [ ]:
df_test['prediction']

0       0
1       1
2       0
3       0
4       0
       ..
3258    0
3259    1
3260    0
3261    0
3262    0
Name: prediction, Length: 3263, dtype: int64

In [ ]:
df_test.head(50)

,text,cleaned_text,prediction
0,Just happened a terrible car crash,"[happened, terrible, car, crash]",0
1,"Heard about #earthquake is different cities, s...","[heard, earthquake, different, cities, stay, s...",1
2,"there is a forest fire at spot pond, geese are...","[forest, fire, spot, pond, geese, fleeing, acr...",0
3,Apocalypse lighting. #Spokane #wildfires,"[apocalypse, lighting, spokane, wildfir]",0
4,Typhoon Soudelor kills 28 in China and Taiwan,"[typhoon, soudelor, kills, china, taiwan]",0
5,We're shaking...It's an earthquake,"[shaking, earthquak]",0
6,They'd probably still show more life than Arse...,"[probably, still, show, life, arsenal, yesterd...",0
7,Hey! How are you?,[hey],0
8,What a nice hat?,"[nice, hat]",0
9,Fuck off!,[fuck],0
